In [2]:
import requests       # Para realizar peticiones a APIs REST
import pandas as pd   # Para manipulación de datos tabulares
import time           # Para hacer pausas entre descargas

In [ ]:
def descargar_datos(base_url, filtro_departamento, nombre_archivo, columna_dep="departamento"):
    """
    Descarga datos desde una API de datos abiertos filtrando por departamento
    y guarda el resultado en un archivo CSV.

    Parámetros:
    - base_url: URL base del recurso en datos.gov.co
    - filtro_departamento: nombre del departamento (ej. "CALDAS")
    - nombre_archivo: nombre del archivo CSV a guardar
    - columna_dep: nombre de la columna que indica el departamento en el dataset
    """

    limit = 50000  # Número máximo de registros por petición
    offset = 0     # Punto de inicio para paginación
    datos = []     # Lista donde se acumulan los bloques descargados

    # Construimos la condición de filtro por departamento
    where_clause = f"upper({columna_dep}) = '{filtro_departamento.upper()}'"

    while True:
        params = {
            "$limit": limit,
            "$offset": offset,
            "$where": where_clause
        }

        print(f"Descargando {nombre_archivo} desde offset={offset}...")
        r = requests.get(base_url, params=params)

        # Verificamos si hubo un error
        if r.status_code != 200:
            print(f"❌ Error {r.status_code} al consultar {nombre_archivo}")
            break

        bloque = r.json()
        if not bloque:
            print(f"✅ Descarga completada para {nombre_archivo}.")
            break

        datos.extend(bloque)
        offset += limit
        time.sleep(1)  # Pausa de 1 segundo

    # Convertimos a DataFrame
    df = pd.DataFrame(datos)

    # Si hay datos, procesamos
    if not df.empty:

        # Filtro adicional por tipo de sensor si aplica
        if "descripcionsensor" in df.columns:
            if "presion" in nombre_archivo:
                df = df[df["descripcionsensor"].str.lower().str.contains("presión", na=False)]
            elif "precipitacion" in nombre_archivo:
                df = df[df["descripcionsensor"].str.lower().str.contains("precipitación", na=False)]
            elif "temperatura" in nombre_archivo:
                df = df[df["descripcionsensor"].str.lower().str.contains("temperatura", na=False)]

        # Conversión de columnas numéricas
        for col in ["valorobservado", "latitud", "longitud", "promedio"]:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")

        # Conversión de columnas temporales
        for col in ["fechaobservacion", "fechas_horas_del_m_ximo", "fechas_horas_del_m_nimo"]:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors="coerce")

        # Guardamos en CSV
        print(f"✅ {len(df)} registros descargados para {nombre_archivo}")
        df.to_csv(nombre_archivo, index=False)

    else:
        print(f"⚠️ No se descargaron datos para {nombre_archivo}.")

    return df


In [ ]:
# Direcciones de los conjuntos de datos (APIs de datos abiertos de Colombia)
URL_PRESION = "https://www.datos.gov.co/resource/62tk-nxj5.json"
URL_PRECIPITACION = "https://www.datos.gov.co/resource/s54a-sgyg.json"
URL_TEMPERATURA = "https://www.datos.gov.co/resource/sbwg-7ju4.json"
URL_CALIDAD_AIRE = "https://www.datos.gov.co/resource/kekd-7v7h.json"


In [ ]:
# Descargamos presión atmosférica para Caldas
df_presion = descargar_datos(URL_PRESION, "CALDAS", "presion_atmosferica_caldas.csv")
# Descargamos precipitaciones para Caldas
df_lluvia = descargar_datos(URL_PRECIPITACION, "CALDAS", "precipitacion_caldas.csv")
# Descargamos temperatura para Caldas
df_temp = descargar_datos(URL_TEMPERATURA, "CALDAS", "temperatura_aire_caldas.csv")
# Descargamos calidad del aire para Caldas (nota: nombre de la columna de departamento cambia)
df_calidad = descargar_datos(URL_CALIDAD_AIRE, "CALDAS", "calidad_aire_caldas.csv", columna_dep="nombre_del_departamento")


In [3]:
# Leer archivos CSV ya descargados
df_presion = pd.read_csv("presion_atmosferica_caldas.csv")
df_lluvia = pd.read_csv("precipitacion_caldas.csv")
df_temp = pd.read_csv("temperatura_aire_caldas.csv")
df_calidad = pd.read_csv("calidad_aire_caldas.csv")

# Mostrar las primeras filas de cada DataFrame
print("📊 Presión atmosférica:")
display(df_presion.head())

print("🌧️ Precipitación:")
display(df_lluvia.head())

print("🌡️ Temperatura:")
display(df_temp.head())

print("🏭 Calidad del aire:")
display(df_calidad.head())


📊 Presión atmosférica:


,codigoestacion,codigosensor,fechaobservacion,valorobservado,nombreestacion,departamento,municipio,zonahidrografica,latitud,longitud,descripcionsensor,unidadmedida
0,26155240,255,2009-02-28 12:00:00,703.2,PNN NEVADOS - AUT,CALDAS,VILLAMARIA,CAUCA,4.859444,-75.382417,Presión Atmosferica (1h),HPa
1,26155240,255,2018-06-10 02:00:00,663.6,PNN NEVADOS - AUT,CALDAS,VILLAMARIA,CAUCA,4.859444,-75.382417,Presión Atmosferica (1h),HPa
2,26155240,255,2011-03-18 13:00:00,703.1,PNN NEVADOS - AUT,CALDAS,VILLAMARIA,CAUCA,4.859444,-75.382417,Presión Atmosferica (1h),HPa
3,26155240,255,2008-08-16 13:00:00,704.1,PNN NEVADOS - AUT,CALDAS,VILLAMARIA,CAUCA,4.859444,-75.382417,Presión Atmosferica (1h),HPa
4,26155220,255,2019-06-09 16:00:00,813.1,VILLAMARIA - AUT,CALDAS,VILLAMARIA,CAUCA,5.048667,-75.513889,Presión Atmosferica (1h),HPa


🌧️ Precipitación:


,codigoestacion,codigosensor,fechaobservacion,valorobservado,nombreestacion,departamento,municipio,zonahidrografica,latitud,longitud,descripcionsensor,unidadmedida
0,26155502,240,2023-09-19 20:30:00,0.0,CENICAFE,CALDAS,MANIZALES,CAUCA,4.991111,-75.597497,PRECIPITACIÓN,mm
1,26165501,240,2023-09-19 20:20:00,0.0,EL CIPRES,CALDAS,SALAMINA,CAUCA,5.433333,-75.500000,PRECIPITACIÓN,mm
2,26165501,240,2023-09-20 09:35:00,0.0,EL CIPRES,CALDAS,SALAMINA,CAUCA,5.433333,-75.500000,PRECIPITACIÓN,mm
3,23025501,240,2023-09-20 00:05:00,0.0,MANZANARES,CALDAS,MANZANARES,MEDIO MAGDALENA,5.163914,-75.117408,PRECIPITACIÓN,mm
4,26135503,240,2023-09-20 09:25:00,0.0,NARANJAL,CALDAS,CHINCHINA,CAUCA,4.971944,-75.652219,PRECIPITACIÓN,mm


🌡️ Temperatura:


,codigoestacion,codigosensor,fechaobservacion,valorobservado,nombreestacion,departamento,municipio,zonahidrografica,latitud,longitud,descripcionsensor,unidadmedida
0,2302500064,68,2024-09-12 04:00:00,15.4,MANZANARES,CALDAS,MANZANARES,MEDIO MAGDALENA,5.265056,-75.143056,TEMPERATURA DEL AIRE A 2 m,°C
1,26155170,68,NaN,11.5,TESORITO FINCA,CALDAS,MANIZALES,CAUCA,5.032222,-75.438333,TEMPERATURA DEL AIRE A 2 m,°C
2,26155110,68,NaN,14.5,AEROPUERTO LA NUBIA,CALDAS,MANIZALES,CAUCA,5.029778,-75.469917,TEMPERATURA DEL AIRE A 2 m,°C
3,26155170,68,NaN,11.4,TESORITO FINCA,CALDAS,MANIZALES,CAUCA,5.032222,-75.438333,TEMPERATURA DEL AIRE A 2 m,°C
4,26155110,68,NaN,12.0,AEROPUERTO LA NUBIA,CALDAS,MANIZALES,CAUCA,5.029778,-75.469917,TEMPERATURA DEL AIRE A 2 m,°C


🏭 Calidad del aire:


,id_estacion,autoridad_ambiental,estaci_n,latitud,longitud,variable,unidades,tiempo_de_exposici_n_horas,a_o,promedio,...,fechas_horas_del_m_ximo,m_nimo,fechas_horas_del_m_nimo,d_as_de_excedencias,c_digo_del_departamento,nombre_del_departamento,c_digo_del_municipio,nombre_del_municipio,tipo_de_estaci_n,ubicacion
0,8990,CORPOCALDAS,LICEO ISABEL LA CATÓLICA,5.068207,-75.511017,PST,ugm3,24,2011,56.3,...,2011-11-26,24.4,2011-10-30,0,17,CALDAS,17001.0,MANIZALES,Fija,"{'type': 'Point', 'coordinates': [-75.511017, ..."
1,99999,CORPOCALDAS,MALTERÍA,5.039227,-75.428200,PST,ugm3,24,2011,99.9,...,2011-06-15,34.8,2011-04-22,0,17,CALDAS,17001.0,MANIZALES,Fija,"{'type': 'Point', 'coordinates': [-75.4282, 5...."
2,8234,CORPOCALDAS,MILÁN,5.046763,-75.480285,PST,ugm3,24,2011,57.0,...,2011-11-05,20.3,2011-06-06,0,17,CALDAS,17001.0,MANIZALES,Fija,"{'type': 'Point', 'coordinates': [-75.480285, ..."
3,99999,CORPOCALDAS,LICORERA,5.068541,-75.517201,PM2.5,ugm3,24,2011,13.5,...,2011-09-23,2.9,NaN,0,17,CALDAS,17001.0,MANIZALES,Fija,"{'type': 'Point', 'coordinates': [-75.517201, ..."
4,99999,CORPOCALDAS,LICORERA,5.068541,-75.517201,PM10,ugm3,24,2011,21.5,...,2011-09-08,7.1,2011-04-23,0,17,CALDAS,17001.0,MANIZALES,Fija,"{'type': 'Point', 'coordinates': [-75.517201, ..."
